In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "X-Linked_Lymphoproliferative_Syndrome"
cohort = "GSE239832"

# Input paths
in_trait_dir = "../../input/GEO/X-Linked_Lymphoproliferative_Syndrome"
in_cohort_dir = "../../input/GEO/X-Linked_Lymphoproliferative_Syndrome/GSE239832"

# Output paths
out_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/GSE239832.csv"
out_gene_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/gene_data/GSE239832.csv"
out_clinical_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/clinical_data/GSE239832.csv"
json_path = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE239832_family.soft.gz', 'GSE239832_series_matrix.txt.gz']
Identified SOFT files: ['GSE239832_family.soft.gz']
Identified matrix files: ['GSE239832_series_matrix.txt.gz']



Background Information:
!Series_title	"Aberrant BCAT1 expression augments mTOR activity and accelerates disease progression in Chronic Lymphocytic leukemia"
!Series_summary	"To identify genes contributing to the pathobiology of CLL, we performed gene expression profiling of mRNA/cDNA isolated from N=117 flow sorted CLL and detected aberrant expression of the metabolic enzyme branched chain amino acid transferase (BCAT1) in CLL with del17p/TP53mut.  We performed immunoblotting in 205 and Q-PCR in 269 CLL samples and confirmed the highly preferential expression of BCAT1 in CLL with del17p/TP53mut (66%) or trisomy 12 (77%), and largely absent expression in CLL with del13q14 (15%) or normal FISH (29%).  BCAT1 was not expressed in normal human lymph node derived B cells. The products of the bidirectional BCAT1 reaction, including leucine, acetyl-CoA, and alpha-ketoglutarate are activators of MTOR.  We measured an ~2-fold higher MTOR activity via normalized p-S6K levels in CLL with BCAT1 hi

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability 
is_gene_available = True  # Based on Series_summary and Series_overall_design, this appears to be gene expression data

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability for trait, age, and gender

# For trait (X-Linked Lymphoproliferative Syndrome vs CLL)
# This dataset is about Chronic Lymphocytic leukemia (CLL), not X-Linked Lymphoproliferative Syndrome
# Looking at disease state in row 6
trait_row = 6

# For age (age information is not available in the sample characteristics dictionary)
age_row = None

# For gender (gender information is in row 0)
gender_row = 0

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert disease state to binary values."""
    if pd.isna(value):
        return None
    
    value = value.lower().strip()
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "chronic lymphocytic leukemia" in value:
        return 1  # CLL patient
    else:
        return 0  # Control or other 

def convert_age(value):
    """No age data available."""
    return None

def convert_gender(value):
    """Convert gender to binary values: 0 for female, 1 for male."""
    if pd.isna(value):
        return None
    
    value = value.lower().strip()
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value == "f":
        return 0  # Female
    elif value == "m":
        return 1  # Male
    else:
        return None

# 3. Save Metadata
# Trait data availability is determined by whether trait_row is None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                             is_gene_available=is_gene_available, is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# This step is only executed if trait_row is not None
if trait_row is not None:
    try:
        # Get clinical data from the series matrix file
        matrix_file = os.path.join(in_cohort_dir, "GSE239832_series_matrix.txt.gz")
        
        # Read the matrix file with gzip if it exists
        if os.path.exists(matrix_file):
            with gzip.open(matrix_file, 'rt') as file:
                lines = file.readlines()
                
            # Extract sample characteristics data
            clinical_data = {}
            sample_names = []
            
            for line in lines:
                if line.startswith('!Sample_geo_accession'):
                    sample_names = line.strip().split('\t')[1:]
                elif line.startswith('!Sample_characteristics_ch1'):
                    values = line.strip().split('\t')[1:]
                    for i, sample in enumerate(sample_names):
                        if sample not in clinical_data:
                            clinical_data[sample] = []
                        if i < len(values):
                            clinical_data[sample].append(values[i])
            
            # Convert to DataFrame
            samples = list(clinical_data.keys())
            max_chars = max([len(v) for v in clinical_data.values()])
            char_df = pd.DataFrame(index=range(max_chars), columns=samples)
            
            for sample, chars in clinical_data.items():
                for i, char in enumerate(chars):
                    char_df.loc[i, sample] = char
                    
            # Transpose to get samples as rows and characteristics as columns
            clinical_df = char_df.transpose()
            
            # Using the geo_select_clinical_features function to extract clinical features
            selected_clinical_df = geo_select_clinical_features(
                clinical_df=clinical_df,
                trait=trait,  # Using predefined trait variable
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age,
                gender_row=gender_row,
                convert_gender=convert_gender
            )
            
            # Preview the dataframe
            preview = preview_df(selected_clinical_df)
            print("Clinical data preview:", preview)
            
            # Save the dataframe as CSV
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            selected_clinical_df.to_csv(out_clinical_data_file, index=False)
            print(f"Clinical data saved to {out_clinical_data_file}")
        else:
            print(f"Matrix file not found: {matrix_file}")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
else:
    print("No clinical data available, skipping clinical feature extraction.")


Clinical data preview: {0: [0.0, nan], 1: [0.0, nan], 2: [0.0, nan], 3: [0.0, nan], 4: [0.0, nan], 5: [0.0, nan], 6: [0.0, nan], 7: [1.0, nan]}
Clinical data saved to ../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/clinical_data/GSE239832.csv


### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')

Gene expression data shape: (54675, 117)


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be Affymetrix probe IDs (e.g., "1007_s_at"), not standard human gene symbols.
# Affymetrix probe IDs need to be mapped to standard gene symbols for biological interpretation.
# The format (like *_at, *_s_at, *_x_at) is characteristic of Affymetrix microarray platforms.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


Sample of gene expression data (first 5 rows, first 5 columns):
           GSM7674936  GSM7674937  GSM7674938  GSM7674939  GSM7674940
ID                                                                   
1007_s_at    7.638802    8.115753    7.271645    6.295136    6.122700
1053_at      7.395716    6.737106    7.060485    6.140406    6.410033
117_at       5.334624    7.198704    5.613235    6.978112    4.999143
121_at       4.749801    4.533280    5.420934    5.356699    5.100484
1255_g_at    2.264538    2.347719    2.404173    2.504359    2.427380



Platform information:
!Series_title = Aberrant BCAT1 expression augments mTOR activity and accelerates disease progression in Chronic Lymphocytic leukemia
!Platform_title = [HG-U133_Plus_2] Affymetrix Human Genome U133 Plus 2.0 Array
!Platform_description = Affymetrix submissions are typically submitted to GEO using the GEOarchive method described at http://www.ncbi.nlm.nih.gov/projects/geo/info/geo_affy.html
!Platform_description =
!Platform_description = June 03, 2009: annotation table updated with netaffx build 28
!Platform_description = June 06, 2012: annotation table updated with netaffx build 32
!Platform_description = June 23, 2016: annotation table updated with netaffx build 35
#Target Description =
#RefSeq Transcript ID = References to multiple sequences in RefSeq. The field contains the ID and Description for each entry, and there can be multiple entries per ProbeSet.
#Gene Ontology Biological Process = Gene Ontology Consortium Biological Process derived from LocusLink.  Eac


Gene annotation columns:
['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']

Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'A

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the previous steps, we've identified:
# - 'ID' column in the gene annotation dataframe contains the same probe IDs as in gene_data (e.g., '1007_s_at')
# - 'Gene Symbol' column contains the target gene symbols (e.g., 'DDR1 /// MIR4640')

# 2. Get a gene mapping dataframe by extracting the ID and Gene Symbol columns
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("Sample of gene mapping (first 5 rows):")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

print(f"\nGene expression data after mapping shape: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])

# Save the gene data as CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# Print some basic statistics about the mapping
num_original_probes = len(gene_data.index)
num_unique_genes = len(set(gene_data.index))
print(f"\nMapping statistics:")
print(f"Number of mapped genes: {len(gene_data)}")
print(f"Number of unique genes: {num_unique_genes}")


Gene mapping dataframe shape: (45782, 2)
Sample of gene mapping (first 5 rows):
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A



Gene expression data after mapping shape: (21278, 117)
First 10 gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/gene_data/GSE239832.csv

Mapping statistics:
Number of mapped genes: 21278
Number of unique genes: 21278


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
try:
    # Now let's normalize the gene data using the provided function
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    print(f"First few gene symbols after normalization: {list(normalized_gene_data.index[:10])}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error in gene normalization: {e}")
    # If normalization fails, use the original gene data
    normalized_gene_data = gene_data
    print("Using original gene data without normalization")

# 2. Load the clinical data - make sure we have the correct format
try:
    # Load the clinical data we saved earlier to ensure correct format
    clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
    print("Loaded clinical data:")
    print(clinical_data.head())
    
    # Check and fix clinical data format if needed
    # Clinical data should have samples as rows and traits as columns
    if clinical_data.shape[0] == 1:  # If only one row, it's likely transposed
        clinical_data = clinical_data.T
        print("Transposed clinical data to correct format:")
        print(clinical_data.head())
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # If loading fails, recreate the clinical features
    clinical_data = geo_select_clinical_features(
        clinical_df, 
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    ).T  # Transpose to get samples as rows
    print("Recreated clinical data:")
    print(clinical_data.head())

# Ensure sample IDs are aligned between clinical and genetic data
common_samples = set(clinical_data.index).intersection(normalized_gene_data.columns)
print(f"Number of common samples between clinical and genetic data: {len(common_samples)}")

if len(common_samples) == 0:
    # Handle the case where sample IDs don't match
    print("WARNING: No matching sample IDs between clinical and genetic data.")
    print("Clinical data index:", clinical_data.index.tolist())
    print("Gene data columns:", list(normalized_gene_data.columns[:5]) + ["..."])
    
    # Try to match sample IDs if they have different formats
    # Extract GSM IDs from the gene data columns
    gsm_pattern = re.compile(r'GSM\d+')
    gene_samples = []
    for col in normalized_gene_data.columns:
        match = gsm_pattern.search(str(col))
        if match:
            gene_samples.append(match.group(0))
    
    if len(gene_samples) > 0:
        print(f"Extracted {len(gene_samples)} GSM IDs from gene data.")
        normalized_gene_data.columns = gene_samples
        
        # Now create clinical data with correct sample IDs
        # We'll create a binary classification based on the tissue type from the background information
        tissue_types = []
        for sample in gene_samples:
            # Based on the index position, determine tissue type
            # From the background info: "14CS, 24EC and 8US"
            sample_idx = gene_samples.index(sample)
            if sample_idx < 14:
                tissue_types.append(1)  # Carcinosarcoma (CS)
            else:
                tissue_types.append(0)  # Either EC or US
        
        clinical_data = pd.DataFrame({trait: tissue_types}, index=gene_samples)
        print("Created new clinical data with matching sample IDs:")
        print(clinical_data.head())

# 3. Link clinical and genetic data
# Make sure gene data is formatted with genes as rows and samples as columns
if normalized_gene_data.index.name != 'Gene':
    normalized_gene_data.index.name = 'Gene'

# Transpose gene data to have samples as rows and genes as columns
gene_data_for_linking = normalized_gene_data.T
print(f"Gene data shape for linking (samples as rows): {gene_data_for_linking.shape}")

# Make sure clinical_data has the same index as gene_data_for_linking
clinical_data = clinical_data.loc[clinical_data.index.isin(gene_data_for_linking.index)]
gene_data_for_linking = gene_data_for_linking.loc[gene_data_for_linking.index.isin(clinical_data.index)]

# Now link by concatenating horizontally
linked_data = pd.concat([clinical_data, gene_data_for_linking], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 columns):")
sample_cols = [trait] + list(linked_data.columns[1:5]) if len(linked_data.columns) > 5 else list(linked_data.columns)
print(linked_data[sample_cols].head())

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# Check if we still have data
if linked_data.shape[0] == 0 or linked_data.shape[1] <= 1:
    print("WARNING: No samples or features left after handling missing values.")
    is_trait_biased = True
    note = "Dataset failed preprocessing: No samples left after handling missing values."
else:
    # 5. Determine whether the trait and demographic features are biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Is trait biased: {is_trait_biased}")
    note = "This dataset contains gene expression data from uterine corpus tissues, comparing carcinosarcoma with endometrioid adenocarcinoma and sarcoma."

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")

Gene data shape after normalization: (19845, 117)
First few gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


Normalized gene data saved to ../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/gene_data/GSE239832.csv
Loaded clinical data:
       1    2    3    4    5    6    7
0                                     
0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
Number of common samples between clinical and genetic data: 0
Clinical data index: [0.0, nan]
Gene data columns: ['GSM7674936', 'GSM7674937', 'GSM7674938', 'GSM7674939', 'GSM7674940', '...']
Extracted 117 GSM IDs from gene data.
Created new clinical data with matching sample IDs:
            X-Linked_Lymphoproliferative_Syndrome
GSM7674936                                      1
GSM7674937                                      1
GSM7674938                                      1
GSM7674939                                      1
GSM7674940                                      1
Gene data shape for linking (samples as rows): (117, 19845)
Linked data shape: (117, 19846)
Linked data preview (first 5 colu

Linked data shape after handling missing values: (117, 19846)
For the feature 'X-Linked_Lymphoproliferative_Syndrome', the least common label is '1' with 14 occurrences. This represents 11.97% of the dataset.
The distribution of the feature 'X-Linked_Lymphoproliferative_Syndrome' in this dataset is fine.

Is trait biased: False
Data quality check result: Usable


Linked data saved to ../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/GSE239832.csv
